In [1]:
%load_ext fortranmagic
%load_ext cython

d:\windows_utils\miniconda3\envs\cimdev\lib\site-packages\fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


In [2]:
%%fortran

! dam discharge calculation test
subroutine ft_test(vol)
    real*8, intent(out) :: vol
    
    vol = 1.4920d-3 * (306505.0d0 - 306500.0d0)**1.5280d0
end subroutine

In [3]:
ft_test()

0.017449982259160068

In [4]:
def py_test():
    discharge = 0.001492 * (306505.0 - 306500.0)**1.5280
    return discharge

In [5]:
py_test()

0.017449982259160068

In [6]:
%timeit ft_test()

The slowest run took 24.03 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 75.9 ns per loop


In [7]:
%timeit py_test()

The slowest run took 11.49 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 95.2 ns per loop


In [8]:
%%cython
cpdef double cy_test():
    cdef double discharge = 0.001492 * (306505.0 - 306500.0)**1.5280
    return discharge

In [9]:
%timeit cy_test()

The slowest run took 28.60 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 38.2 ns per loop


In [10]:
# import numpy as np
# from libc.math import exp
from math import exp
def py_calc_interim_cmd(cmd, param_d, rainfall):
    """Calculate interim CMD (M_{f}) in its linear form.

    :param cmd: float, current Catchment Moisture Deficit (M_{k})
    :param param_d: float, model parameter factor `d`
    :param rainfall: float, rainfall for current time step in mm

    :returns: float, interim CMD (M_{f})
    """
    if cmd < param_d:
        Mf = cmd * exp(-rainfall / param_d)
    elif cmd < (param_d + rainfall):
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else:
        Mf = cmd - rainfall
    # End if

    return Mf
# End calc_interim_cmd()

In [11]:
%timeit py_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 12.05 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 333 ns per loop


In [12]:
%time py_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.779091697232

In [13]:
%%fortran

subroutine ft_calc_interim_cmd(cmd, param_d, rainfall, Mf)
    real*8 :: cmd, param_d, rainfall
    real*8, intent(out) :: Mf

    if (cmd.lt.param_d) then
        Mf = cmd * exp(-rainfall / param_d)
    else if (cmd.lt.(param_d + rainfall)) then
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else
        Mf = cmd - rainfall
    endif

end subroutine

In [14]:
%timeit ft_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 71.03 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 216 ns per loop


In [15]:
%time ft_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.77909169723196

In [16]:
%%cython

from __future__ import division

from libc.math cimport exp

cpdef double cy_calc_interim_cmd(double cmd, double param_d, double rainfall):
    """Calculate interim CMD (M_{f}) in its linear form.

    :param cmd: float, current Catchment Moisture Deficit (M_{k})
    :param param_d: float, model parameter factor `d`
    :param rainfall: float, rainfall for current time step in mm

    :returns: float, interim CMD (M_{f})
    """
    cdef double Mf
    if cmd < param_d:
        Mf = cmd * exp(-rainfall / param_d)
    elif cmd < (param_d + rainfall):
        Mf = param_d * exp((-rainfall + cmd - param_d) / param_d)
    else:
        Mf = cmd - rainfall
    # End if

    return Mf
# End calc_interim_cmd()

Cannot build msvcr library: "msvcr90d.dll" not found


In [17]:
%timeit cy_calc_interim_cmd(200.0, 194.0, 16.5)

The slowest run took 15.88 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 161 ns per loop


In [18]:
%time cy_calc_interim_cmd(200.0, 194.0, 16.5)

Wall time: 0 ns


183.77909169723196

In [19]:
# %load_ext julia.magic

In [20]:
# %%julia
# 1+1


In [24]:
import math
import numpy as np

In [25]:
%timeit math.pi

The slowest run took 68.71 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 42.5 ns per loop


In [26]:
%timeit np.pi

10000000 loops, best of 3: 37.8 ns per loop


In [30]:
%timeit np.arctan(2.0 * 150.0)

The slowest run took 19.38 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 621 ns per loop


In [31]:
%timeit math.atan(2.0 * 150.0)

The slowest run took 21.50 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 102 ns per loop


In [35]:
%timeit math.tan(150)

The slowest run took 13.58 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 134 ns per loop


In [34]:
%timeit np.tan(150)

The slowest run took 18.34 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 795 ns per loop


In [36]:
def ret_tan(x):
    return math.tan(x)

In [42]:
%timeit ret_tan(150.0)

The slowest run took 13.29 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 192 ns per loop


In [39]:
%%fortran

subroutine ft_tan(x)
    real*8, intent(inout) :: x
            
    x = tan(x)

end subroutine

In [41]:
%timeit ft_tan(150.0)

The slowest run took 18.02 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 142 ns per loop


In [44]:
%%cython
from libc.math cimport tan

cpdef double cy_tan(double x):
    return tan(x)

Cannot build msvcr library: "msvcr90d.dll" not found


In [46]:
%timeit cy_tan(150.0)

The slowest run took 16.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000000 loops, best of 3: 87 ns per loop


In [2]:
%%fortran
subroutine ft_flow_calc(a, b, prev_quick, prev_slow, u, r, area, loss, pscale, quick, slow, flow, a2, b2, tmp_slow, tmp_flow)
    real*8, intent(in) :: a, b, prev_quick, prev_slow, u, r, area, loss, pscale
    real*8, intent(out) :: quick, slow, flow
    real*8, intent(out) :: a2, b2, tmp_flow, tmp_slow
    
    a2=0.5d0
    if ((prev_quick+u*area-0.5*loss).gt.0) then
        quick=1/(1+a)*(prev_quick+u*area-loss/2.0d0)
        flow=a*quick

    else
        if(loss.eq.0.0d0) then
            a2=0.0d0
        else
            a2=(prev_quick+u*area)/loss
            if(a2.gt.1) then
                a2=1.0d0
            endif
            if(a2.lt.0) then
                a2=0.0d0
            endif
        endif

        quick=prev_quick+u*area-a2*loss
        flow=0.0d0
    endif
    
    tmp_flow = flow

    b2=1.0d0-a2
    tmp_slow = (prev_slow+pscale*r*area-loss*b2)
    if ((prev_slow+r*area-b2*loss).gt.0) then
        slow=1/(1+b)*(prev_slow+pscale*r*area-loss*b2)
        flow=flow+b*slow
    else
        slow=slow+pscale*r*area-b2*loss
    endif

end subroutine

In [3]:
# a, b, prev_quick, prev_slow, u, r, area, loss, pscale
a = 54.0
b = 0.18
prev_quick = 24.0
prev_slow = 500.0
u = 0.1
r = 0.01
area = 1900
loss = 0.0
pscale = 0.0

In [4]:
def calc_ft_flows(prev_quick, prev_slow, e_rain, recharge, area, a, b, loss=0.0, pscale=0.0):
    """

    :param prev_quick: float, previous quickflow storage
    :param prev_slow: float, previous slowflow storage
    :param e_rain: float, effective rainfall in mm
    :param recharge: float, recharge amount in mm
    :param area: float, catchment area in km^2
    :param a: float, `a` factor controlling quickflow rate
    :param b: float, `b` factor controlling slowflow rate
    :param loss: float, losses in mm depth

    :returns: tuple[float], quick store, slow store, outflow
    """
    a2 = 0.5
    if (prev_quick + e_rain * area - 0.5 * loss) > 0.0:
        quick_store = 1.0 / (1.0 + a) * (prev_quick + e_rain * area - loss / 2.0)
        outflow = a * quick_store
    else:
        a2 = 0.0 if loss == 0.0 else max(0.0, min(1.0, prev_quick + e_rain * area / loss))
        quick_store = prev_quick + e_rain * area - a2 * loss
        outflow = 0.0
    # End if
    
    tmp_flow = outflow

    b2 = 1.0 - a2
    tmp_slow = (prev_slow+r*area-loss*b2)
    slow_store = prev_slow + recharge * area - loss * b2  # b2 * loss
    if (prev_slow + recharge * area - b2 * loss) > 0.0:
        slow_store = 1.0 / (1.0 + b) * slow_store
        outflow = outflow + b * slow_store
    # End if

    return quick_store, slow_store, outflow, a2, b2, tmp_slow, tmp_flow
# End calc_ft_flows()

In [5]:
ft_flow_calc(a, b, prev_quick, prev_slow, u, r, area, loss, pscale=1.0)  # , 0.0, 0.0

(3.8909090909090907,
 439.83050847457633,
 289.27858243451465,
 0.5,
 0.5,
 519.0,
 210.1090909090909)

In [6]:
calc_ft_flows(prev_quick, prev_slow, u, r, area, a, b, loss)

(3.8909090909090907,
 439.83050847457633,
 289.27858243451465,
 0.5,
 0.5,
 519.0,
 210.1090909090909)

In [7]:
slow_store = prev_slow + r * area - loss * 0.5
slow_store

519.0

In [20]:
from math import exp
def calc_ET(e, evap, interim_cmd, f, d):
    """Calculate evapotranspiration

    :param e: float, temperature to PET conversion factor (a stress threshold)
    :param evap: float, evaporation for given time step.
    :param interim_cmd: float, Catchment Moisture Deficit prior to accounting for ET losses (`M_{f}`)
    :param f: float, calibrated parameter that acts as a multiplication factor on `d`
    :param d: float, flow threshold factor
    """
    et = e * evap
    param_g = f * d
    print(d)
    et = et * min(1.0, exp(2.0 * (1.0 - (interim_cmd / param_g))))

    return et
# End calc_ET()

In [21]:
calc_ET(e=1.0, evap=6.0, interim_cmd=150.0, f=0.5, d=150)

150


0.8120116994196762

In [17]:
%%fortran
subroutine ft_calc_cmd(cmd, f, e, evap, et)
    real*8, intent(in) :: cmd, f, e, evap
    real*8, intent(out) :: et
    if (cmd.gt.f) then
        et = e*evap*exp( (1-cmd/f)*2)
    else
        et = e*evap
    endif
end subroutine

In [23]:
ft_calc_cmd(150.0, 75.0, 1.0, 6.0)

0.8120116994196762